In [43]:
# 1.1 Importar bibliotecas necesarias
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

# 1.2 Definir los nombres de las columnas según la descripción del dataset
columnas = [
    'clase', 'forma_sombrero', 'superficie_sombrero', 'color_sombrero', 
    'magulladuras', 'olor', 'disposicion_branquias', 'espaciado_branquias', 
    'tamano_branquias', 'color_branquias', 'forma_tallo', 'raiz_tallo', 
    'superficie_tallo_anillo', 'superficie_tallo_anillo_arriba', 
    'color_tallo_arriba_anillo', 'color_tallo_debajo_anillo', 'tipo_velo', 
    'color_velo', 'numero_anillo', 'tipo_anillo', 'color_esporas', 
    'poblacion', 'habitat'
]

# 1.3 Cargar el dataset
df = pd.read_csv('../data_mushroom/agaricus-lepiota.data', header=None, names=columnas)

# 1.4 Exploración inicial
print("Dimensiones del dataset:", df.shape)
print("\nPrimeras 5 filas:")
print(df.head())
print("\nInformación del dataset:")
print(df.info())
print("\nEstadísticas descriptivas:")
print(df.describe(include='all'))

Dimensiones del dataset: (8124, 23)

Primeras 5 filas:
  clase forma_sombrero superficie_sombrero color_sombrero magulladuras olor  \
0     p              x                   s              n            t    p   
1     e              x                   s              y            t    a   
2     e              b                   s              w            t    l   
3     p              x                   y              w            t    p   
4     e              x                   s              g            f    n   

  disposicion_branquias espaciado_branquias tamano_branquias color_branquias  \
0                     f                   c                n               k   
1                     f                   c                b               k   
2                     f                   c                b               n   
3                     f                   c                n               n   
4                     f                   w                b          

## Tratamiento de valores faltantes y columnas no informativas

In [44]:
# 2.1 Identificar valores faltantes (representados como '?')
print("Valores '?' por columna:")
for col in df.columns:
    count = (df[col] == '?').sum()
    if count > 0:
        print(f"{col}: {count} valores '?'")

# 2.2 Eliminar la columna 'tipo_velo' si tiene muchos valores faltantes
# o rellenar valores faltantes con la moda
if 'tipo_velo' in df.columns:
    if (df['tipo_velo'] == '?').sum() > len(df) * 0.5:  # Si más del 50% son faltantes
        df = df.drop('tipo_velo', axis=1)
    else:
        # Rellenar con la moda
        moda = df['tipo_velo'].mode()[0]
        df['tipo_velo'] = df['tipo_velo'].replace('?', moda)

# 2.3 Para otras columnas con pocos valores faltantes, rellenar con la moda
for col in df.columns:
    if (df[col] == '?').any():
        moda = df[col].mode()[0]
        df[col] = df[col].replace('?', moda)

# 2.4 Identificar y eliminar columnas con poca varianza (constantes o casi constantes)
columnas_a_eliminar = []
for col in df.columns:
    if df[col].nunique() == 1:  # Si la columna tiene un solo valor único
        columnas_a_eliminar.append(col)
        print(f"Eliminando columna {col} por tener un solo valor único")

df = df.drop(columnas_a_eliminar, axis=1)
print(f"\nColumnas restantes después de la limpieza: {df.columns.tolist()}")

Valores '?' por columna:
raiz_tallo: 2480 valores '?'
Eliminando columna tipo_velo por tener un solo valor único

Columnas restantes después de la limpieza: ['clase', 'forma_sombrero', 'superficie_sombrero', 'color_sombrero', 'magulladuras', 'olor', 'disposicion_branquias', 'espaciado_branquias', 'tamano_branquias', 'color_branquias', 'forma_tallo', 'raiz_tallo', 'superficie_tallo_anillo', 'superficie_tallo_anillo_arriba', 'color_tallo_arriba_anillo', 'color_tallo_debajo_anillo', 'color_velo', 'numero_anillo', 'tipo_anillo', 'color_esporas', 'poblacion', 'habitat']


## Codificación de variables categóricas

In [45]:
# 3.1 Separar la variable objetivo
y = df['clase']  # Guardar la variable objetivo
X = df.drop('clase', axis=1)  # Características

# 3.2 Aplicar One-Hot Encoding a las características
print("\nAplicando One-Hot Encoding...")
X_encoded = pd.get_dummies(X, drop_first=True)
print(f"Dimensiones después de One-Hot Encoding: {X_encoded.shape}")

# 3.3 Codificar la variable objetivo (e=0, p=1)
le = LabelEncoder()
y_encoded = le.fit_transform(y)


Aplicando One-Hot Encoding...
Dimensiones después de One-Hot Encoding: (8124, 94)


## Reduccion de dimensionalidad con PCA

In [46]:
# 4.1 Estandarizar los datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_encoded)

# 4.2 Aplicar PCA
pca = PCA()
X_pca = pca.fit_transform(X_scaled)

# 4.3 Determinar el número óptimo de componentes
# Calcular la varianza explicada acumulada
explained_variance_ratio = np.cumsum(pca.explained_variance_ratio_)

# Graficar la varianza explicada
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(explained_variance_ratio) + 1), explained_variance_ratio, 'b-')
plt.axhline(y=0.95, color='r', linestyle='--', label='95% de varianza explicada')
plt.xlabel('Número de Componentes')
plt.ylabel('Varianza Explicada Acumulada')
plt.title('Varianza Explicada por Componentes Principales')
plt.legend()
plt.grid(True)
plt.savefig('../data_mushroom/pca_variance.png')
plt.close()

# 4.4 Seleccionar el número de componentes que explican el 95% de la varianza
n_components = np.argmax(explained_variance_ratio >= 0.95) + 1
print(f"\nNúmero óptimo de componentes para explicar el 95% de la varianza: {n_components}")

# 4.5 Aplicar PCA con el número óptimo de componentes
pca = PCA(n_components=n_components)
X_pca = pca.fit_transform(X_scaled)


Número óptimo de componentes para explicar el 95% de la varianza: 55


## Aplicación de K-Means Clustering

In [47]:
# 5.1 Determinar el número óptimo de clusters usando el método del codo
inertias = []
K = range(1, 11)

for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_pca)
    inertias.append(kmeans.inertia_)

# Graficar el método del codo
plt.figure(figsize=(10, 6))
plt.plot(K, inertias, 'bx-')
plt.xlabel('Número de Clusters (k)')
plt.ylabel('Inercia')
plt.title('Método del Codo para Determinar k Óptimo')
plt.grid(True)
plt.savefig('../data_mushroom/elbow_method.png')
plt.close()

# 5.2 Aplicar K-Means con k=2 (sabemos que hay 2 clases: comestible y venenoso)
kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X_pca)

# 5.3 Visualizar los clusters en el espacio de las dos primeras componentes principales
plt.figure(figsize=(10, 8))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=clusters, cmap='viridis', alpha=0.7)
plt.xlabel('Primera Componente Principal')
plt.ylabel('Segunda Componente Principal')
plt.title('Clustering de Hongos con K-Means')
plt.colorbar(scatter, label='Cluster')
plt.savefig('../data_mushroom/kmeans_clustering.png')
plt.close()

# 5.4 Comparar clusters con las clases reales (solo para evaluación, en aprendizaje no supervisado real no tendríamos esta información)
cluster_map = pd.DataFrame()
cluster_map['clase_real'] = y_encoded
cluster_map['cluster'] = clusters

# Mapear clusters a las clases mayoritarias
cluster_to_class = {}
for cluster in cluster_map['cluster'].unique():
    mask = cluster_map['cluster'] == cluster
    # La clase mayoritaria en este cluster
    majority_class = cluster_map[mask]['clase_real'].mode()[0]
    cluster_to_class[cluster] = majority_class

cluster_map['cluster_asignado'] = cluster_map['cluster'].map(cluster_to_class)

# Calcular precisión del clustering
accuracy = (cluster_map['clase_real'] == cluster_map['cluster_asignado']).mean()
print(f"\nPrecisión del clustering K-means: {accuracy*100:.2f}%")


Precisión del clustering K-means: 86.80%


## Comparación con un Modelo Supervisado (Random Forest)

In [48]:
# 6.1 Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y_encoded, test_size=0.3, random_state=42, stratify=y_encoded
)

# 6.2 Entrenar un modelo Random Forest
print("\nEntrenando modelo Random Forest...")
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# 6.3 Hacer predicciones en el conjunto de prueba
y_pred = rf.predict(X_test)

# 6.4 Evaluar el modelo
print("\nRendimiento del modelo Random Forest:")
print(classification_report(y_test, y_pred))
print(f"Exactitud: {accuracy_score(y_test, y_pred)*100:.2f}%")

# 6.5 Visualizar la importancia de las características
feature_importance = pd.DataFrame({
    'feature': X_encoded.columns,
    'importance': rf.feature_importances_
}).sort_values('importance', ascending=False)

# Mostrar las 10 características más importantes
plt.figure(figsize=(12, 8))
sns.barplot(x='importance', y='feature', data=feature_importance.head(10))
plt.title('Top 10 Características Más Importantes (Random Forest)')
plt.tight_layout()
plt.savefig('../data_mushroom/feature_importance.png')
plt.close()


Entrenando modelo Random Forest...

Rendimiento del modelo Random Forest:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1263
           1       1.00      1.00      1.00      1175

    accuracy                           1.00      2438
   macro avg       1.00      1.00      1.00      2438
weighted avg       1.00      1.00      1.00      2438

Exactitud: 100.00%


## Visualización de Resultados

In [49]:
# 7.1 Visualización de la distribución de clases en los clusters
plt.figure(figsize=(10, 6))
sns.countplot(x='cluster', hue='clase_real', data=cluster_map)
plt.title('Distribución de Clases Reales en Cada Cluster')
plt.xlabel('Cluster')
plt.ylabel('Cantidad')
plt.legend(title='Clase', labels=['Comestible (e)', 'Venenoso (p)'])
plt.savefig('../data_mushroom/cluster_class_distribution.png')
plt.close()

# 7.2 Matriz de confusión para el modelo supervisado
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=le.classes_)
disp.plot(cmap='Blues')
plt.title('Matriz de Confusión - Random Forest')
plt.savefig('../data_mushroom/confusion_matrix.png')
plt.close()

## Conclusiones

In [50]:
# %% [markdown]
# ## 7. Conclusiones

# %%
# Verificar y definir todas las variables necesarias
import os
import pandas as pd
import numpy as np
from sklearn.metrics import silhouette_score, davies_bouldin_score, accuracy_score, classification_report

# Inicializar variables con valores por defecto
columnas_constantes = []
n_components = 0
explained_variance_ratio = np.array([0])
accuracy = 0
silhouette = 0
davies_bouldin = 0
y_test, y_pred = None, None

# Verificar y calcular columnas constantes si df_clean existe
if 'df_clean' in locals() or 'df_clean' in globals():
    columnas_constantes = [col for col in df_clean.columns if df_clean[col].nunique() == 1]
    num_columnas = len(df_clean.columns)
else:
    num_columnas = "No disponible (df_clean no definido)"

# Verificar y calcular métricas de PCA si es necesario
if 'X_scaled' in locals() or 'X_scaled' in globals():
    if 'pca' not in locals() and 'pca' not in globals():
        from sklearn.decomposition import PCA
        pca = PCA(n_components=0.95)
        X_pca = pca.fit(X_scaled)
        n_components = pca.n_components_
        explained_variance_ratio = np.cumsum(pca.explained_variance_ratio_)
    else:
        n_components = pca.n_components_
        explained_variance_ratio = np.cumsum(pca.explained_variance_ratio_)

# Verificar y calcular métricas de clustering si es posible
if 'clusters' in locals() or 'clusters' in globals():
    if 'X_pca' in locals() or 'X_pca' in globals():
        try:
            silhouette = silhouette_score(X_pca, clusters)
            davies_bouldin = davies_bouldin_score(X_pca, clusters)
            
            # Calcular precisión del clustering si tenemos y_encoded
            if 'y_encoded' in locals() or 'y_encoded' in globals():
                cluster_map = pd.DataFrame({'clase_real': y_encoded, 'cluster': clusters})
                cluster_to_class = {cluster: cluster_map[cluster_map['cluster'] == cluster]['clase_real'].mode()[0] 
                                  for cluster in cluster_map['cluster'].unique()}
                cluster_map['cluster_asignado'] = cluster_map['cluster'].map(cluster_to_class)
                accuracy = (cluster_map['clase_real'] == cluster_map['cluster_asignado']).mean()
        except Exception as e:
            print(f"Advertencia: No se pudieron calcular todas las métricas de clustering: {e}")

# Imprimir resumen de resultados
print("="*50)
print("RESUMEN DE RESULTADOS")
print("="*50)

print("\n1. Preprocesamiento:")
if 'df' in locals() or 'df' in globals():
    tipo_velo_info = "tipo_velo" if 'tipo_velo' in df.columns else "tipo_velo (no encontrada)"
    print(f"- Columnas eliminadas: {columnas_constantes + [tipo_velo_info] if columnas_constantes else 'Ninguna'}")
else:
    print("- No se pudo acceder al dataframe original (df no definido)")

print(f"- Columnas restantes: {num_columnas}")

if 'X_encoded' in locals() or 'X_encoded' in globals():
    print(f"- Tamaño después de One-Hot Encoding: {X_encoded.shape[1]} características")
else:
    print("- Tamaño después de One-Hot Encoding: No disponible (X_encoded no definido)")

print(f"\n2. Reducción de Dimensionalidad (PCA):")
print(f"- Número óptimo de componentes para 95% de varianza: {n_components}")
if len(explained_variance_ratio) > 0:
    print(f"- Varianza explicada acumulada: {explained_variance_ratio[-1]:.2%}")
else:
    print("- Varianza explicada acumulada: No disponible")

print(f"\n3. Resultados de Clustering (K-means):")
if 'accuracy' in locals() or 'accuracy' in globals():
    print(f"- Precisión del clustering: {accuracy*100:.2f}%")
    print(f"- Puntuación de Silueta: {silhouette:.4f}")
    print(f"- Índice de Davies-Bouldin: {davies_bouldin:.4f}")
else:
    print("- No se pudieron calcular las métricas de clustering. Asegúrate de haber ejecutado el clustering.")

print(f"\n4. Rendimiento del Modelo Supervisado (Random Forest):")
if 'y_test' in locals() and 'y_pred' in locals() and y_test is not None and y_pred is not None:
    print(f"- Exactitud: {accuracy_score(y_test, y_pred)*100:.2f}%")
    print(f"- Reporte de clasificación:\n{classification_report(y_test, y_pred, target_names=['Comestible (e)', 'Venenoso (p)'])}")
else:
    print("- No se ejecutó la evaluación del modelo supervisado o hay datos faltantes.")

print("\n5. Archivos generados:")
try:
    data_dir = os.path.join("..", "data_mushroom")
    if os.path.exists(data_dir):
        generated_files = [f for f in os.listdir(data_dir) 
                         if f.endswith(('.png', '.jpg', '.jpeg', '.csv')) 
                         and os.path.isfile(os.path.join(data_dir, f))]
        if generated_files:
            for file in generated_files:
                print(f"- {file}: {os.path.join('data_mushroom', file)}")
        else:
            print("- No se encontraron archivos generados en el directorio data_mushroom/")
    else:
        print(f"- El directorio {data_dir} no existe.")
except Exception as e:
    print(f"- Error al buscar archivos generados: {e}")

print("\n¡Análisis completado con éxito!")

RESUMEN DE RESULTADOS

1. Preprocesamiento:
- Columnas eliminadas: Ninguna
- Columnas restantes: No disponible (df_clean no definido)
- Tamaño después de One-Hot Encoding: 94 características

2. Reducción de Dimensionalidad (PCA):
- Número óptimo de componentes para 95% de varianza: 55
- Varianza explicada acumulada: 95.46%

3. Resultados de Clustering (K-means):
- Precisión del clustering: 86.80%
- Puntuación de Silueta: 0.1203
- Índice de Davies-Bouldin: 2.8815

4. Rendimiento del Modelo Supervisado (Random Forest):
- No se ejecutó la evaluación del modelo supervisado o hay datos faltantes.

5. Archivos generados:
- confusion_matrix.png: data_mushroom/confusion_matrix.png
- feature_importance.png: data_mushroom/feature_importance.png
- elbow_method.png: data_mushroom/elbow_method.png
- kmeans_clustering.png: data_mushroom/kmeans_clustering.png
- cluster_class_distribution.png: data_mushroom/cluster_class_distribution.png
- pca_variance.png: data_mushroom/pca_variance.png

¡Análisis c